In [ ]:
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import powerlaw

## Reading and converting data format

We start by reading the airports file to a pandas Dataframe.

In [ ]:
airportsDF = pd.read_csv('airports.dat', header=None, names=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

From there, we create a directed graph and add all the airports as nodes to it (except airports with no IATA identifier, which are ignored).
This is because we will use this identifiers later to associate this data with the routes data.

In [ ]:
G = nx.DiGraph()

for index, row in airportsDF.iterrows():
    if row['IATA'] == '\\N':
        continue
    G.add_node(row['IATA'], pos=(row['lat'],row['lon']))

Our nodes are identified by their IATA identifiers and have a corresponding position (corresponding to their real geographic location) 

In [ ]:
print(G.nodes['LIS'])

So far we have a graph with nodes but no edges

In [ ]:
print("Node count: " + str(len(G.nodes)))
print("Edge count: " + str(len(G.edges)))

Now we read the routes data and add each one as an edge of our graph previously created
This routes are directed, meaning that if an airline operates services from A to B and from B to A, both A-B and B-A are listed as diferent edges
We also add weight to this edges, the distance between airports in km.

In [ ]:
routesDF = pd.read_csv('routes.dat', header=None, names=['airline', 'airlineID', 'source', 'sourceID', 'dest', 'destID', 'codeshare', 'stops', 'equipment'])

for index, row in routesDF.iterrows():
    if(row['source'] not in G.nodes or row['dest'] not in G.nodes):
        continue
    dist = geopy.distance.distance((G.nodes[row['source']]['pos']), (G.nodes[row['dest']]['pos'])).km
    G.add_edge(row['source'], row['dest'], weight=dist)

As an example, we take a route from Lisbon to Cologne to show this representation.

In [ ]:
print(list(G.edges('LIS',data=True))[0])

We can now see that our graph has nodes and edges.

In [ ]:
print("Node count: " + str(len(G.nodes)))
print("Edge count: " + str(len(G.edges)))

## Cleaning data

We now "clean" our graph, removing the nodes which have either 0 incoming flights or 0 outgoing flights (or both).
We iterate the graph until no more changes are made. This is because after removing one node, another one might lose the only incoming or outgoing route it had previously.

In [ ]:
i=0
while i == 0:
  i = 1
  H = G.copy()
  for node in H.in_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])
      
  H = G.copy()
  for node in H.out_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])

We end up with a significant reduction in nodes and lose barely any edges, obtaining what we wanted and removing unutilized airports from the network.

In [ ]:
print("Node count: " + str(len(G.nodes)))
print("Edge count: " + str(len(G.edges)))

## Analysing network metrics

We can also see that the Frankfurt airport, Charles de Gaulle airport in Paris and Amesterdam are the nodes with the highest degree, with around 450 incoming or outgoing routes each.

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)
print(nodes[0:3])

We also see that the majority of airports have a very low degree, resulting in a near zero density in our graph.

In [ ]:
# Graph density
nx.density(G)

With the degree distribution we verify that almost 800 airports (of the near 3000) have a degree of only 2, meaning only one incoming route and one outgoing one.

In [ ]:
# Degree Distribution
degrees = [G.degree(n) for n in G.nodes()]
plt.hist(degrees, bins=np.logspace(np.log10(2),np.log10(max(degrees)), 20))
plt.xscale("log")
plt.title("Histogram of Degree Distribution")
plt.ylabel("Node Count")
plt.xlabel("Degree Value")


The cumulative distribution also proves this point.

In [ ]:
import collections

degrees = [G.degree(n) for n in G.nodes()]
degree_sequence = sorted(degrees, reverse=True) # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
cs = np.cumsum(cnt)
plt.loglog(deg, cs, 'bo')
plt.title("Cumulative Distribution plot")
plt.ylabel("Sample with value > Degree")
plt.xlabel("Degree")
plt.show()

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

degrees = [G.degree(n) for n in G.nodes()]
degree_counts = Counter(degrees)
print(degree_counts)

x, y = zip(*degree_counts.items())

plt.xlabel('Degree')
plt.xscale('log')

plt.ylabel('Node count')
plt.yscale('log')

plt.scatter(x, y, marker='.')
plt.show()

## Degree centrality

We now calculate some metrics of the network, starting with the degree centrality
As seen before, Frankfurt is the airport with the highest degree.
A military base in Greenland is the one of the many airports with only degree 2, resulting in a low degree centrality.

In [ ]:
# compute graph degree centrality
dc = nx.degree_centrality(G)

max_dc = max(dc, key=dc.get)
print("Airport with the max degree centrality: " + max_dc)
print("Value: " + str(dc[max_dc]))

min_dc = min(dc, key=dc.get)
print("Airport with the min degree centrality: " + min_dc)
print("Value: " + str(dc[min_dc]))

## Eigenvector centrality

For the eigenvector centrality we get that Amsterdam airport has the highest value.

In [ ]:
# compute eigen vector centrality
ec = nx.eigenvector_centrality(G)

max_ec = max(ec, key=ec.get)
print("Airport with the max eigenvector centrality: " + max_ec)
print("Value: " + str(ec[max_ec]))

min_ec = min(ec, key=ec.get)
print("Airport with the min eigenvector centrality: " + min_ec)
print("Value: " + str(ec[min_ec]))

## Closeness centrality

For the closeness centrality we get that Frankfurt airport has the highest value.

In [ ]:
# compute closeness centrality
cc = nx.closeness_centrality(G, distance='weight')

max_cc = max(cc, key=cc.get)
print("Airport with the max closeness centrality: " + max_cc)
print("Value: " + str(cc[max_cc]))

min_cc = min(cc, key=cc.get)
print("Airport with the min closeness centrality: " + min_cc)
print("Value: " + str(cc[min_cc]))

## Harmonic centrality

In [ ]:
# compute harmonic centrality
hc = nx.harmonic_centrality(G, distance='weight')

max_hc = max(hc, key=hc.get)
print("Airport with the max harmonic centrality: " + max_hc)
print("Value: " + str(hc[max_hc]))

min_hc = min(hc, key=hc.get)
print("Airport with the min harmonic centrality: " + min_hc)
print("Value: " + str(hc[min_hc]))

In [ ]:
# sort cc and hc by dictionary key
cc = dict(sorted(cc.items()))
hc = dict(sorted(hc.items()))

# Correlation matrix between Closerness Centrality and Harmonic Centrality
cc_list = list(cc.values())
hc_list = list(hc.values())
# normalize
cc_list = (cc_list - np.mean(cc_list))/np.std(cc_list)
hc_list = (hc_list - np.mean(hc_list))/np.std(hc_list)
print(np.corrcoef(cc_list, hc_list))
print(cc_list)
print(hc_list)
# plot scatter plot
plt.scatter(cc_list, hc_list)
plt.xlabel("Closeness Centrality values (normalized)")
plt.ylabel("Harmonic Centrality values (normalized)")
m, b = np.polyfit(cc_list, hc_list, 1)
plt.plot(cc_list, m*cc_list + b , color='red', label="Linear Regression")
plt.legend()

## Betweenness centrality

For the betweenness centrality we get that Charles de Gaulle airport has the highest value.

In [ ]:
# compute betweenness centrality
bc = nx.betweenness_centrality(G)

max_bc = max(bc, key=bc.get)
print("Airport with the max betweenness centrality: " + max_bc)
print("Value: " + str(bc[max_bc]))

min_bc = min(bc, key=bc.get)
print("Airport with the min betweenness centrality: " + min_bc)
print("Value: " + str(bc[min_bc]))

# sort the betweenness centrality
sorted_bc = sorted(bc.items(), key=lambda x: x[1], reverse=True)
print(sorted_bc[0:5])

## Page rank

For the page rank we get that Charles de Gaulle airport has the highest value.

In [ ]:
# compute pagerank
pr = nx.pagerank(G)

max_pr = max(pr, key=pr.get)
print("Airport with the max page rank: " + max_pr)
print("Value: " + str(pr[max_pr]))

min_pr = min(pr, key=pr.get)
print("Airport with the min page rank: " +min_pr)
print("Value: " + str(pr[min_pr]))

## Clustering

For the clustring we get that Goroka airport has the highest value. 

In [ ]:
# compute clustering coefficient
cl = nx.clustering(G)

max_cl = max(cl, key=cl.get)
print("Airport with the max clustering: " + max_cl)
print("Value: " + str(cl[max_cl]))

min_cl = min(cl, key=cl.get)
print("Airport with the min clustering: " + min_cl)
print("Value: " + str(cl[min_cl]))

### Strongly connected components

Using the networkx function to get us the strongly connected components, we verify that out graph has 7.
One of them has almost every node and the rest are small, due to this we'll only consider the biggest one in some future steps.

In [ ]:
# generate strongly connected components
sccs = nx.strongly_connected_components(G)
print([len(Gc) for Gc in sccs])

sccs = nx.strongly_connected_components(G)
sccs_graphs = (G.subgraph(c) for c in sccs)

largest_scc = list(sccs_graphs)[0]
print(len(largest_scc))

## Classifying regime of network

In [ ]:
import math

degrees = G.degree()
print(degrees)
sum_of_degrees = 0
for x in degrees:
    sum_of_degrees += x[1]

degree_average = sum_of_degrees / len(degrees)
print("degree average: " + str(degree_average))

log_degree = math.log(degree_average)
log_n = math.log(len(degrees))

print("log n: " + str(log_n))
print("log degree: " + str(log_degree))


print("log n / log degree: " + str(log_n / log_degree))

Before having a strongly connected component, calculating the average shortest path and the diameter of the network was not possible since there were some infinite distances, now we can do it.

## Average shortest path lenght and diameter

In [ ]:
avg_path_length = nx.average_shortest_path_length(largest_scc)
print(avg_path_length)

In [ ]:
nx.diameter(largest_scc)

In [ ]:
dists = dict(nx.all_pairs_shortest_path_length(G))

In [ ]:
print(dists['LIS'])
seen_nodes = []
teste = {}
for node in G.nodes():
    seen_nodes.append(node)
    
    for k, v in dists[node].items():
        if k not in seen_nodes:
            teste[v] = teste.get(v, 0) + 1
            
print(teste)  

In [ ]:
print(len(teste.values()))
print(list(teste.values()))


In [ ]:
x_list = list(range(1,13))
y_list = list(teste.values())
y_list_norm = []
size = sum(list(teste.values()))
for y in y_list:
    y_list_norm.append(y / size)

cumulative = np.cumsum(y_list_norm)
point = 0.9 * sum(y_list_norm)

plt.plot(x_list, [point]*len(x_list), "--")
plt.plot([5]*len(x_list), cumulative, "--")
plt.plot(x_list, cumulative, label="CDF")
plt.plot(x_list, y_list_norm, color="red", label="PDF") 
plt.legend()

## Plotting the data on a map

We can now plot each airport in a map and give each dot some coloring and size according to its centrality.
Having created a function that receives which centrality to be used, the list of nodes to be plotted and a threshold (values lower that this threshold won't be displayed), we can plot the maps for each of the centralities calculated before.

In [ ]:
def plotMap(label, centrality_dict, list_nodes, threshold):
    newAirportsDF = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])
    
    # get max and min to normalize
    max_value = max(centrality_dict.values())
    min_value = min(centrality_dict.values())

    # from the list given, add the nodes that have a higher centrality value than the threshold
    for node in list_nodes:
        if(centrality_dict[node] > threshold):
            x = airportsDF.loc[airportsDF['IATA'] == node]
            x_norm = (centrality_dict[node] - min_value) / (max_value - min_value)
            x['color'] = centrality_dict[node]
            x['markersize'] = (x_norm **4) * 200
            newAirportsDF = pd.concat([newAirportsDF, x])

    geometry = [Point(xy) for xy in zip(newAirportsDF['lon'], newAirportsDF['lat'])]
    gdf = GeoDataFrame(newAirportsDF, geometry=geometry)

    # #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    gdf.plot(column="color", ax=world.plot(figsize=(20, 12)), marker='o', markersize="markersize", cmap='rainbow')

In [ ]:
# Ignores a warning that is not worrying in this case and creates a lot of noisy output
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Degree centrality map

In [ ]:
plotMap("Degree centrality", dc, G.nodes, 0)

## Eigenvector centrality map

In [ ]:
plotMap("Eigenvector centrality", ec, G.nodes, 0)

## Closeness centrality map

In [ ]:
plotMap("Closeness centrality", cc, G.nodes, 0)

## Betweenness centrality map

In [ ]:
plotMap("Betweenness centrality", bc, G.nodes, 0)

## Page rank map

In [ ]:
plotMap("Page rank", pr, G.nodes, 0)

## Assortativity

In [ ]:
print(nx.algorithms.assortativity.degree_pearson_correlation_coefficient(G))

In [ ]:
min_val, max_val = min(G.degree, key = lambda d: d[1])[1], max(G.degree, key = lambda d: d[1])[1]

mapping = {x: x for x in range(max_val//9)}

matrix = nx.algorithms.assortativity.degree_mixing_matrix(G, mapping=mapping)

f = plt.figure(figsize=(19, 15))
plt.matshow(matrix, fignum=f.number)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

## Average of neighbors' degrees

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1])
i=0
degree = nodes[0][1]

degrees = [degree]
average_list = [[],]

for node in nodes:
    if node[1] != degree:
        degree=node[1]
        degrees.append(degree)
        average_list[i] = int(sum(average_list[i])/len(average_list[i]))
        average_list.append([])
        i+=1
    for neig in nx.all_neighbors(G, node[0]):
        average_list[i].append(G.degree[neig])

average_list[i] = int(sum(average_list[i])/len(average_list[i]))
xpoints = np.array(degrees)
ypoints = np.array(average_list)

plt.xlabel("Degree")
plt.ylabel("Average degree of neighbors")
plt.plot(xpoints, ypoints)
plt.show()

## Communities

In [ ]:
D = G.copy()
print(len(D.nodes))
print(len(D.edges))
i=0
while i == 0:
  i = 1
  H = D.copy()
  for node in H.in_degree:
    if node[1] < 5:
      i = 0
      D.remove_node(node[0])
      
  H = D.copy()
  for node in H.out_degree:
    if node[1] < 5:
      i = 0
      D.remove_node(node[0])

print(len(D.nodes))
nodes = sorted(D.degree, key=lambda x: x[1])
print(nodes)
print(len(D.edges))

### Greedy algorithm

In [ ]:
g = nx.algorithms.community.greedy_modularity_communities(D, resolution=2)
print(len(g))

In [ ]:
from random import randint

newAirportsDF = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

#generate a random color for each community and plot them in the map
for community in g:
    rand_color = randint(0,80)
    for node in community:
        x = airportsDF.loc[airportsDF['IATA'] == node]
        x['color'] = rand_color
        newAirportsDF = pd.concat([newAirportsDF, x])

geometry = [Point(xy) for xy in zip(newAirportsDF['lon'], newAirportsDF['lat'])]
gdf = GeoDataFrame(newAirportsDF, geometry=geometry)

# this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(column="color", ax=world.plot(figsize=(20, 12)), marker='o', markersize=8, cmap='gist_ncar')

### Louvain method (best method)

In [ ]:
bp = nx.algorithms.community.louvain_communities(D)
print(len(bp))
print(bp)

In [ ]:
from random import randint

newAirportsDF = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

#generate a random color for each community and plot them in the map
for community in bp:
    rand_color = randint(0,80)
    for node in community:
        x = airportsDF.loc[airportsDF['IATA'] == node]
        x['color'] = rand_color
        newAirportsDF = pd.concat([newAirportsDF, x])

geometry = [Point(xy) for xy in zip(newAirportsDF['lon'], newAirportsDF['lat'])]
gdf = GeoDataFrame(newAirportsDF, geometry=geometry)

# this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(column="color", ax=world.plot(figsize=(20, 12)), marker='o', markersize=8, cmap='gist_ncar')

## Fitting degree distribution to a power law function

In [ ]:
degree_sequence = sorted([d for n, d in G.degree()], reverse=True) # used for degree distribution and powerlaw test

 # Power laws are probability distributions with the form:p(x)∝x−α
fit = powerlaw.Fit(degree_sequence)

print(fit.power_law.alpha)

fig2 = fit.plot_pdf(color='b', linewidth=2)
fit.power_law.plot_pdf(color='g', linestyle='--', ax=fig2)

## Robustness analysis

In [ ]:
#about 7 minutes to run
max = len(G.nodes())
x = []
y = []

strongest_connection = 3190

nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)
for number_removed in range(max):
    F = G.copy()
    number_of_edges = []
    

    for i in nodes[0: number_removed]:
        F.remove_node(i[0])

    sccs = sorted(nx.strongly_connected_components(F), key=lambda t: len(t), reverse=True)
    x += [number_removed/max]
    y += [len(sccs[0])/strongest_connection]


print(x)
print(y)

In [ ]:
#about 6 minutes to run
from random import shuffle
max2 = len(G.nodes())
x2 = []
y2 = []

strongest_connection2 = 3190


nodes = list(G.degree)
shuffle(nodes)

for number_removed in range(max2):
    F = G.copy()
    number_of_edges = []
    

    for i in nodes[0: number_removed]:
        F.remove_node(i[0])

    sccs = sorted(nx.strongly_connected_components(F), key=lambda t: len(t), reverse=True)
    x2 += [number_removed/max2]
    y2 += [len(sccs[0])/strongest_connection2]


print(x2)
print(y2)

In [ ]:
#about 7 minutes to run
max3 = len(G.nodes())
x3 = []
y3 = []

strongest_connection3 = 3190

bc = nx.betweenness_centrality(G)

nodes = sorted(bc.items(), key=lambda x: x[1], reverse=True)

for number_removed in range(max3):
    F = G.copy()
    number_of_edges = []
    

    for i in nodes[0: number_removed]:
        F.remove_node(i[0])

    sccs = sorted(nx.strongly_connected_components(F), key=lambda t: len(t), reverse=True)
    x3 += [number_removed/max3]
    y3 += [len(sccs[0])/strongest_connection3]


print(x3)
print(y3)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x3, y3)
plt.xlabel('Number of removed nodes')
plt.ylabel('SCCs')
plt.title('')
plt.show()

## Studying the SI Model

In [ ]:
def plotMapSI(infected_dic, list_nodes):
    newAirportsDF = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

    # from the list given, add the nodes that have a higher centrality value than the threshold
    for node in list_nodes:
            x = airportsDF.loc[airportsDF['IATA'] == node]
            x['color'] = 1 if infected_dic[node] else 0
            x['markersize'] = 10
            newAirportsDF = pd.concat([newAirportsDF, x])

    geometry = [Point(xy) for xy in zip(newAirportsDF['lon'], newAirportsDF['lat'])]
    gdf = GeoDataFrame(newAirportsDF, geometry=geometry)

    # #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    gdf.plot(column="color", ax=world.plot(figsize=(20, 12)), marker='o', markersize="markersize", cmap='bwr')

In [ ]:
def plotSI(infected, susceptible):
    days = list(range(len(infected)))
    plt.figure(2)
    plt.plot(days, infected, label="Infected", color="red")
    plt.plot(days, susceptible, label="Susceptible", color="blue")
    plt.xlabel("Days")
    plt.ylabel("Number of people")
    plt.legend()
    plt.margins(x=0, y=0)
    plt.grid()

In [ ]:
# compute SI step
def SI_step(network):
    for node in network.nodes:
        if network.nodes[node]['infected'] and network.nodes[node]['active']:
            for neighbor in network.neighbors(node):
                if not network.nodes[neighbor]['infected']:
                    if np.random.random() < network.nodes[node]['beta']:
                        network.nodes[neighbor]['infected'] = True

# compute SI model
def SI_model(network, dc_sorted, nSteps, starterNode, theDay, airportPercentage):
    susceptible = []
    infected = []
    network.nodes[starterNode]['infected'] = True
    for step in range(nSteps):
        # remove nodes based on degree centrality at the Day
        if(step == theDay):
            # get the top 20% of nodes based on degree centrality
            closedAirports = dc_sorted[0:int(len(dc_sorted)*airportPercentage)]
            # remove the nodes that are not in the top 20%
            for node in network.nodes:
                if node in closedAirports:
                    network.nodes[node]['beta'] = 0.0001

        susceptible.append(len([node for node in network.nodes if not network.nodes[node]['infected']]))
        infected.append(len([node for node in network.nodes if network.nodes[node]['infected']]))
        SI_step(network)
    
    return susceptible, infected

def full_SI_model(graph, degree_centrality, nSteps, starterNode, theDay, airportPercentage):
    network = graph.copy()
    # get largest strongly connected component
    sccs = nx.strongly_connected_components(network)
    sccs_graphs = (network.subgraph(c) for c in sccs)
    network = list(sccs_graphs)[0]

    # sorted degree centrality
    dc_sorted = sorted(degree_centrality, key=dc.get, reverse=True)

    # add infection and recovery rate to nodes
    for node in network.nodes:
        network.nodes[node]['beta'] = 0.01
        network.nodes[node]['infected'] = False
        network.nodes[node]['active'] = True

    susceptible, infected = SI_model(network, dc_sorted, nSteps, starterNode, theDay, airportPercentage)
    return susceptible, infected, network

In [ ]:
# model without preventive measures
susceptible, infected, network = full_SI_model(G, dc, 100, 'LIS', 0, 0)
days = list(range(len(infected)))

infected_dic = {node:network.nodes[node]['infected'] for node in network.nodes}

print(infected[len(infected)-1])
print(susceptible[len(susceptible)-1])

# model with preventive measures
susceptible_prev, infected_prev, network_prev = full_SI_model(G, dc, 100, 'LIS', 7, 0.02)
infected_dic_prev = {node:network_prev.nodes[node]['infected'] for node in network_prev.nodes}

plt.figure(1)
plt.plot(days, infected, label="Infected", color="red")
plt.plot(days, susceptible, label="Susceptible", color="blue")
plt.plot(days, infected_prev, label="Infected (prev. measures)", color="red", linestyle='dashed')
plt.plot(days, susceptible_prev, label="Susceptible (prev. measures)", color="blue", linestyle='dashed')
plt.xlabel("Days")
plt.ylabel("Number of people")
plt.legend()
plt.margins(x=0, y=0)
plt.grid()

plt.figure(2)
plt.plot(days, infected, label="Infected", color="red")
plt.plot(days, susceptible, label="Susceptible", color="blue")
plt.xlabel("Days")
plt.ylabel("Number of people")
plt.legend()
plt.margins(x=0, y=0)
plt.grid()

plt.figure(3)
plt.plot(days, infected_prev, label="Infected", color="red")
plt.plot(days, susceptible_prev, label="Susceptible", color="blue")
plt.xlabel("Days")
plt.ylabel("Number of people")
plt.legend()
plt.margins(x=0, y=0)
plt.grid()

In [ ]:
print(infected_prev[len(infected)-1])
print(susceptible_prev[len(susceptible)-1])